In [10]:
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from IPython import embed

In [11]:
class ReIDNet(nn.Module):
    def __init__(self, num_classes, loss={'softmax, metric'}, **kwargs):
        super().__init__()
        orinet=torchvision.models.alexnet(pretrained=True)
        self.loss=loss
        self.basenet=nn.Sequential(*list(orinet.children())[0:1])
        self.linear=nn.Linear(256, 2048)
        self.classifier=nn.Linear(2048, num_classes)
    def forward(self, x):
        x=self.basenet(x)
        cnn_result=F.avg_pool2d(x, x.size()[2:]).view(x.size(0),-1)
        f=self.linear(cnn_result)
        #归一化
        f=1.*f / (torch.norm(f,2,dim=-1, keepdim=True).expand_as(f)+1e-12)
        #以下if/else用于控制网络训练与测试所使用的网络结构
        if not self.training:
            return f
        if self.loss=={'softmax'}:
            y=self.classifier(f)
            return y
        elif self.loss=={'metric'}:
            return f
        elif self.loss=={'softmax, metric'}:
            y=self.classifier(f)
            return y,f
        else:
            print('loss setting error')

In [ ]:
if __name__=='__main__':
    model=ReIDNet(num_classes=751)
    imgs=torch.rand(32,3,128,64)
    f=model(imgs)
    print(imgs.size())
    print(f.size())
    embed()

torch.Size([32, 3, 128, 64])
torch.Size([32, 751])
Python 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
Type 'copyright', 'credits' or 'license' for more information
IPython 7.19.0 -- An enhanced Interactive Python. Type '?' for help.

In [1]: model.train
Out[1]: 
<bound method Module.train of ReIDNet(
  (basenet): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace)
      (10): Conv2d(256, 256, kernel_size=(3